In [ ]:
# 파이썬 동적 웹 크롤링 라이브러리
from selenium.webdriver.common.by import By
from selenium import webdriver
# 알림창 등을 회피하기 위한 키보드 라이브러리
import pyautogui

# 각종 라이브러리
import pandas as pd
import time
import re

# 유적지 데이터셋 읽기 및 유적지명 리스트 생성
df = pd.read_csv('KC_507_LLR_HISTST_2022_test.csv')
dest_lst = df['prompt'].values.tolist()

results = {}

# 크롤링 시작
driver = webdriver.Chrome()
driver.get('https://www.instagram.com')
time.sleep(3)

# id, pw 입력 및 로그인
ID = 'id'
pw = 'pw'

# ID 문자열 전송
input_id = driver.find_elements(By.CSS_SELECTOR, 'input')[0]
input_id.clear()
input_id.send_keys(ID)

time.sleep(0.5)

# PW 문자열 전송
input_pw = driver.find_elements(By.CSS_SELECTOR, 'input')[1]
input_pw.clear()
input_pw.send_keys(pw)

# 로그인
time.sleep(2)
input_pw.submit()
time.sleep(10)

# 팝업창 닫기
pyautogui.press('tab')
time.sleep(1)
pyautogui.press('enter')
time.sleep(3)
pyautogui.press('esc')
time.sleep(1)

# 검색창 열기
driver.find_elements(By.CSS_SELECTOR,'[aria-label="검색"]')[0].click()
time.sleep(1)
input_sch = driver.find_elements(By.CSS_SELECTOR, 'input')[0]

#7627개 유적지 순회
for i in range(len(dest_lst)):
    
    # 500개 검색 후 세션 해제 방지를 위한 페이지 새로고침
    if i%500 == 0 and i!=0:
        driver.refresh()
        time.sleep(10)
        driver.find_elements(By.CSS_SELECTOR,'[aria-label="검색"]')[0].click()
        time.sleep(1)
        input_sch = driver.find_elements(By.CSS_SELECTOR, 'input')[0]
    
    # 유적지명 검색
    word = dest_lst[i]
    input_sch.clear()
    input_sch.send_keys(word)
    time.sleep(1)
    
    # 결과 HTML 별도 TXT 파일로 저장
    html = driver.page_source
    with open('./html_save/'+str(word)+'.txt','w',encoding = 'utf16') as f:
        temp = str(html)
        # 용량을 줄이기 위해 불필요한 부분 제거 후 저장
        temp = temp[temp.find('프로필 사진'):temp.find('회원님을 위한 추천')]
        f.write(temp)

    print(word, ' Complete')
    time.sleep(1)


In [ ]:
# 분석용 라이브러리 
import pandas as pd
# 파일시스템 제어용 라이브러리
import os

# html 저장 위치 접근 및 DataFrame 준비
path_dir = './html_save/'
file_list = os.listdir(path_dir)
df = pd.DataFrame({'SITE':[], 'TAGS':[]})

# txt 파일로 저장해둔 html 순회
for index, name in enumerate(file_list):
    
    # 파일 데이터 문자열로 읽어오기
    f=open('./html_save/'+name,"rt",encoding = 'utf16')
    tmp = ''
    while True:
        line = f.readline()
        tmp = tmp+line
        if line == '' :
            break
    f.close()

    # 문자열에서 게시물 개수를 탐색
    h_site = name.replace('.txt','')
    count = 0
    start = tmp.find('>#'+h_site+'<')
    if start>=0:
        pnt1 = tmp[start:].find('게시물')
        pnt2 = tmp[start+pnt1:].find('<span>')
        end = tmp[start+pnt1+pnt2:].find('</span>')
        hashtags = tmp[pnt1+pnt2+start:pnt1+pnt2+start+end].replace('<span>','')
        
        # 태그 수 만개 이상, 한글표시 숫자로 변경
        ten_thou = hashtags.find('만')
        if ten_thou < 0:
            count = int(hashtags)
        else:
            count = int(float(hashtags.replace('만',''))*10000)
        
    # DataFrame에 유적지명 | 해시태그 수 로 저장
    print(h_site, count)
    df.loc[index, 'SITE'] = h_site
    df.loc[index, 'TAGS'] = count
    
display(df)

df.to_csv('Insta_Analyze.csv', index=False, encoding='utf16')

In [ ]:
# 분석 및 시각화 라이브러리
import pandas as pd
import matplotlib.pyplot as plt

# 해시태그 데이터셋 불러오기
insta = pd.read_csv('insta_tags.csv', encoding='cp949')
insta['TAGS'] = insta['TAGS'].astype('long')

# 태그수가 0인 데이터와 0 초과인 데이터 분리
df = insta[insta['TAGS']>0]
df0 = insta[insta['TAGS']==0]

# 0초과 데이터셋을 6개로 데이터 수 균등 분할함
labels = [1,2,3,4,5,6]
df2 = pd.qcut(df['TAGS'],6,labels = labels)
#분할 경계값 확인
print(df2.unique())


# 분할된 데이터셋의 개수를 확인
temp = pd.value_counts(df2)
print(temp)

# 태그가 0인 데이터의 개수 확인
print(df0.count())

# 히스토그램을 통해 균등한 분할이 이루어졌는지 확인
values = [df0['TAGS'].count(), temp[1], temp[2], temp[3], temp[4], temp[5], temp[6]]
labels = [0,1,2,3,4,5,6]
plt.bar(labels, values)
plt.show()